In [20]:
import numpy as np

In [36]:
A = np.random.random(20)
B = np.random.random(20)

tcc = np.random.random(20)

In [45]:
order = 1
X = np.random.random((20-order, order))

In [46]:
for temp_order in range(1, order+1):   
    
    remove_from_end = len(tcc) - (order - temp_order)

    a = A[:-order]
    b = B[slice(temp_order, remove_from_end)]
    
    bo = [element.astype(int) == temp_order for element in (b-a).astype('timedelta64[h]') ]
    print(bo)
    ins = b.copy()
    ins[np.array(bo)] = np.nan
    X[:, temp_order - 1] = ins


[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]


In [ ]:


def dataset_to_numpy_order_traditional_ar(dataset, order, bias = True):
    """ Tranforms a dataset to matrices.

    Parameters
    ----------------------------
    dataset : xr.Dataset
        Contains the data you want to make a prediction based.
    order : float
        The number of previos timesteps included as predictors.
    bias : bool
        Determines weather to include a bias column or not (default True)
    keep the order of xarray time, lat, lon

    Returns
    ---------------------
    X : array-like
        Matrix containing the explanatory variables.
    y : array-like
        Responce variable.

    Notes
    --------------------------
    Index description:

    5 (4) - tcc previos time step

    """

    if bias:
        var_index = 1
    else:
        var_index = 0

    times = dataset.time.values
    #print("Detected {} samples.".format(len(times)))
    X = np.zeros( (len(times)-order, order + var_index))
    y = np.zeros( (len(times)-order ))

    tcc = dataset.tcc.values
    print('len tcc {}'.format(len(tcc)))

    if bias:
        X[:, 0] = 1 # bias

    y = tcc[:len(times)-order, np.newaxis]
    print('len y should be tcc - order {}'.format(len(y)))
    # tcc1, tcc2, ..., tcc_n
    for temp_order in range(1, order+1):
        a = times[:len(times)-temp_order]
        b = times[temp_order:]
        bo = [element.astype(int) == temp_order for element in (b-a).astype('timedelta64[h]') ]
        print('len bo {}'.format(len(bo)))
        remove_from_end = order - temp_order
        if remove_from_end != 0:
            # remove_from_end = 1
            # Which clouds to add at which column, remember that they shoudl start from t-1, t-2, t-3 ...
            X[:, var_index] = tcc[temp_order:][bo][:-remove_from_end]
        else:
            X[:, var_index] = tcc[temp_order:][bo]
        var_index+=1
    #print(X.shape)
    #print(y.shape)
    return X, y

In [5]:
A = np.random.random((744, 81, 161, 5))

In [6]:
len(A.shape)

4

In [8]:
import xarray as xr

In [12]:
dataset = xr.open_dataset('/home/hanna/lagrings/ERA5_monthly/2012_04_tcc.nc')

In [15]:
tcc = dataset.tcc.values

In [28]:
order = 1
bias = False

if bias:
    var_index = 1
else:
    var_index = 0

times = dataset.time.values
#print("Detected {} samples.".format(len(times)))
X = np.zeros( (len(times)-order, order + var_index))
y = np.zeros( (len(times)-order ))

tcc = dataset.tcc.sel(latitude = 30, longitude = 0).values
print('len tcc {}'.format(len(tcc)))

if bias:
    X[:, 0] = 1 # bias

y = tcc[:len(times)-order, np.newaxis]
print('len y should be tcc - order {}'.format(len(y)))
# tcc1, tcc2, ..., tcc_n
for temp_order in range(1, order+1):
    a = times[:len(times)-temp_order]
    b = times[temp_order:]
    bo = [element.astype(int) == temp_order for element in (b-a).astype('timedelta64[h]') ]
    print('len bo {}'.format(len(bo)))
    remove_from_end = order - temp_order
    
    if remove_from_end != 0:
        # remove_from_end = 1
        # Which clouds to add at which column, remember that they shoudl start from t-1, t-2, t-3 ...
        X[:, var_index] = tcc[temp_order:][bo][:-remove_from_end]
    else:
        X[:, var_index] = tcc[temp_order:][bo]
    var_index+=1

len tcc 720
len y should be tcc - order 719
len bo 719


In [44]:
order = 3
bias = False

tcc = dataset.tcc.sel(latitude = 30, longitude = 0).values

if bias:
    var_index = 5
else:
    var_index = 4

times = dataset.time.values
#print("Detected {} samples.".format(len(times)))
X = np.zeros( (len(times)-order, order + var_index))
y = np.zeros( (len(times)-order ))

q   = dataset.tcc.sel(latitude = 30, longitude = 0).values#dataset.q.values
t2m = dataset.tcc.sel(latitude = 30, longitude = 0).values#dataset.t2m.values
r   = dataset.tcc.sel(latitude = 30, longitude = 0).values#dataset.r.values
sp  = dataset.tcc.sel(latitude = 30, longitude = 0).values#dataset.sp.values
tcc = dataset.tcc.sel(latitude = 30, longitude = 0).values#dataset.tcc.values

X[:, 0] = q[:len(times)-order]
X[:, 1] = t2m[:len(times)-order]
X[:, 2] = r[:len(times)-order]
X[:, 3] = sp[:len(times)-order]

if bias:
    X[:, 4] = 1 # bias

y = tcc[:len(times)-order, np.newaxis]

# tcc1, tcc2, ..., tcc_n
for temp_order in range(1, order+1):
    a = times[:len(times)-temp_order]
    #print(len(a))
    b = times[temp_order:]
    #print(len(b))

    remove_from_end = order - temp_order
    if remove_from_end != 0:
        a = times[:len(times)-temp_order]
        #print(len(a))
        b = times[temp_order:]
        bo = [element.astype(int) == temp_order for element in (b-a).astype('timedelta64[h]')]
        #print(len(bo[:-remove_from_end]))
        # remove_from_end = 1
        # Which clouds to add at which column, remember that they shoudl start from t-1, t-2, t-3 ...
        X[:, var_index] = tcc[temp_order:][bo][:-remove_from_end]
    else:
        a = times[:len(times)-temp_order]
        #print(len(a))
        b = times[temp_order:]
        bo = [element.astype(int) == temp_order for element in (b-a).astype('timedelta64[h]') ]
        X[:, var_index] = tcc[temp_order:][bo]
    var_index+=1
X.shape

(717, 7)